[View in Colaboratory](https://colab.research.google.com/github/Kaliostrogoblin/PDD/blob/master/examples/Train_Siamese_Network.ipynb)

In [0]:
import numpy as np

np.random.seed(13)

In [3]:
!rm -r -f PDD
!git clone https://github.com/Kaliostrogoblin/PDD.git

Cloning into 'PDD'...
remote: Counting objects: 117, done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 117 (delta 62), reused 78 (delta 27), pack-reused 0
Receiving objects: 100% (117/117), 22.29 KiB | 7.43 MiB/s, done.
Resolving deltas: 100% (62/62), done.


In [0]:
import os
os.chdir('../')

In [4]:
import os
os.chdir('PDD')
os.listdir()

['.gitignore', 'examples', 'README.md', '.git', 'pdd']

In [5]:
from pdd.datasets.grape import load_data

train_data_path, test_data_path = load_data(split_on_train_test=True, random_state=13)

Using TensorFlow backend.
100%|██████████████████████| 5/5 [00:00<00:00, 48.99it/s]

Splitting on train and test...


In [0]:
from pdd.models import get_feature_extractor
from pdd.models import make_siamese
import keras.backend as K
import tensorflow as tf

sess = tf.Session()
K.set_session(sess)

In [7]:
input_shape = (256, 256, 3)

print("Building feature extractor...")
feature_extractor = get_feature_extractor(input_shape)

print("Constructing siamese network...")
siams = make_siamese(feature_extractor, dist='l1', loss='contrastive')
siams.summary()

Building feature extractor...
Constructing siamese network...
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
model_1 (Model)                 (None, 1024)         14902496    input_2[0][0]                    
                                                                 input_3[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 1)      

In [0]:
from pdd.utils.training import SiameseBatchGenerator

In [0]:
train_batch_gen = SiameseBatchGenerator.from_directory(dirname=train_data_path)
test_batch_gen = SiameseBatchGenerator.from_directory(dirname=test_data_path)

def siams_generator(batch_gen, batch_size=None):
    while True:
        batch_xs, batch_ys = batch_gen.next_batch(batch_size)
        yield [batch_xs[0], batch_xs[1]], batch_ys

In [0]:
siams.fit_generator(
    generator=siams_generator(train_batch_gen),
    steps_per_epoch=100,
    epochs=50,
    verbose=1,
    validation_data=siams_generator(test_batch_gen),
    validation_steps=30,
    shuffle=True
    #callbacks=[
     #   EarlyStopping(min_delta=1e-2, patience=7, mode='min', verbose=1),
      #  ReduceLROnPlateau(factor=0.2, patience=5, mode='min', verbose=1),
      #  ModelCheckpoint(callback_path, save_weights_only=True, mode='min', verbose=1)]
)

Epoch 1/50
100/100 [==============================] - 76s 763ms/step - loss: 2329.9938 - acc: 0.0338 - val_loss: 3.1066 - val_acc: 0.1042
Epoch 2/50
 31/100 [========>.....................] - ETA: 41s - loss: 2.0634 - acc: 0.2621

In [0]:
from pdd.models import TfKNN
from pdd.utils.data_utils import create_dataset_from_dir

from sklearn.metrics import accuracy_score

In [0]:
train_dataset = create_dataset_from_dir(train_data_path, shuffle=True)
test_dataset = create_dataset_from_dir(test_data_path, shuffle=True)

In [0]:
tfknn = TfKNN(sess, feature_extractor, (train_dataset['data'], train_dataset['target']))

In [28]:
preds = tfknn.predict(test_dataset['data'], return_dist=True)
accuracy = accuracy_score(test_dataset['target'], preds[0])
print("Accuracy: %.2f" % accuracy)

Accuracy: 0.30
